In [1]:
import pandas as pd

In [3]:
df=pd.read_csv('spotify.csv')

In [7]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [8]:
df.tail()

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [10]:
df.shape

(57650, 4)

In [11]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [12]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [13]:
df.head(10)

,artist,song,text
0,Chaka Khan,Tell Me Something Good,You ain't got no kind of feeling inside \r\nI...
1,Slayer,War Zone,Explosive you feel the impact hit your face \...
2,Counting Crows,Come Pick Me Up,(Original by Ryan Adams) \r\n \r\nWhen they ...
3,Widespread Panic,Imitation Leather Shoes,My little brother is an insect \r\nHe likes t...
4,Olly Murs,Head To Toe,"One, two, three, now I'm building up the coura..."
5,John McDermott,The Bard Of Armagh,Oh list to the strains of a poor Irish harper ...
6,Enrique Iglesias,Love 4 Fun,So you think \r\nYou got it all worked out \...
7,W.A.S.P.,Hate To Love Me,You know I deceived ya \r\nYou see it's all i...
8,Grand Funk Railroad,Walk Like A Man,Walk like a man \r\nAnd talk like a man \r\n...
9,Toto,Blackeye,Hey come old Blackeye \r\nHe's looking for me...


In [14]:
df['text'][0]

"You ain't got no kind of feeling inside  \r\nI got something that will sure 'nuff set your stuff on fire  \r\nYou refuse to put anything before your pride  \r\nI got something that will knock all your pride aside  \r\n  \r\nTell me something good (tell me, tell me, tell me)  \r\nTell me that you love me  \r\nTell me something good (tell me, tell me, tell me)  \r\nTell me that you like it, yeah  \r\n  \r\nGot no time is what you're known to say  \r\nI'll make you wish there were 48 hours to each day  \r\nProblem is you ain't been loved like you should  \r\nWhat I got to give will sure 'nuff do you good\r\n\r\n"

In [15]:
df.shape

(5000, 3)

In [16]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [17]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [18]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [21]:
similarity[0]

array([1.        , 0.02880577, 0.07009617, ..., 0.07470784, 0.03564487,
       0.05585552])

In [23]:
df[df['song'] == 'Walk Like A Man']

,artist,song,text
8,Grand Funk Railroad,Walk Like A Man,walk like a man and talk like a man walk like ...


In [24]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [26]:
recommendation('Walk Like A Man')

['My Way',
 'R.u. Man Enough',
 'Dead Man',
 'One Man Woman',
 'Mannish Boy',
 'Stand By Your Man',
 'A Man Without A Dream',
 "I'm Your Man",
 'Walks Like A Lady',
 'Rock Me, Baby',
 'Deep In The Motherlode',
 'Man Of The Year',
 "(Feels Like I'm) Walking On Water",
 'Great King Rat',
 'Better As A Memory',
 'Love Man',
 'There Goes My Baby',
 "Lovin' Man",
 "Lady's Man",
 "Rock 'n' Roll Over You"]

In [27]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))